In [1]:
import pandas as pd
import re

In [2]:
trim_tail_number = lambda s : None if not s else re.sub("^[A-Z]", "", s)

In [3]:
# Create a mapping dictionary for manufacturer names
manufacturer_mapping = {
    'AIRBUS.*': 'AIRBUS',  # Will match any variation starting with AIRBUS
    'BOEING.*': 'BOEING',
	'MCDONNELL DOUGLAS.*': 'MCDONNELL DOUGLAS',
    'LOCKHEED.*': 'LOCKHEED MARTIN',
    'EMBRAER.*': 'EMBRAER',
	'SAAB.*': 'SAAB',
	'DIAMOND AIRCRAFT.*': 'DIAMOND AIRCRAFT',
	'ROBINSON.*': 'ROBINSON',
	'AIR TRACTOR.*': 'AIR TRACTOR',
}

# Function to standardize manufacturer names
def standardize_manufacturer(name : str) -> str:
    for pattern, standard_name in manufacturer_mapping.items():
        if pd.notna(name) and re.match(pattern, name, flags=re.IGNORECASE):
            return standard_name
    return name.strip()  # Return original if no match

In [4]:

years = list(range(2018, 2023))
for year in years:
    _faa_df = pd.read_parquet(f"./data/faa_{year}.parquet")

    _delay_df = pd.read_parquet(f"./data/delay_{year}.parquet")

    # Remove preceeding identificaton letters from tail 
    # numbers, since the FAA doesn't include them
    _delay_df["Tail_Number"] = _delay_df["Tail_Number"].apply(trim_tail_number)

    # Truncate manufacturer names
    _faa_df['MFR'] = _faa_df['MFR'].apply(standardize_manufacturer)

    # Trim excess from delay dataframe
    _delay_df = _delay_df[[
        'Cancelled', 
        'Diverted', 
        'DepDelayMinutes', 
        'ArrDelayMinutes',
		'Tail_Number',
		'Month'
        ]]

    # Join with FAA
    _clean_df = _delay_df.merge(
        _faa_df,
        left_on='Tail_Number',
        right_on='N-NUMBER',
        how='inner',
        suffixes=('__DELAY__', '__FAA__')
    )

    print(f"original: {len(_delay_df)} rows | joined: {len(_clean_df)} rows")
    _clean_df.to_parquet(f"./data/clean_joined_{year}.parquet")

original: 5689512 rows | joined: 5502497 rows
original: 8091684 rows | joined: 7803194 rows
original: 5022397 rows | joined: 4695086 rows
original: 6311871 rows | joined: 6021567 rows
original: 4078318 rows | joined: 3913835 rows


~~Yikes. That's one hell of a reduction.~~ Fixed!

In [5]:
clean_2018_df = pd.read_parquet("./data/clean_joined_2018.parquet")
clean_2018_df.describe()

,DepDelayMinutes,ArrDelayMinutes,Month,YEAR MFR,SPEED,TYPE-ACFT,NO-SEATS,NO-ENG,AGE
count,5.438374e+06,5.422618e+06,5.502497e+06,5.502497e+06,5.502497e+06,5.502497e+06,5.502497e+06,5.502497e+06,5.502497e+06
mean,1.344554e+01,1.366774e+01,7.290793e+00,2.006321e+03,4.365800e-02,4.997466e+00,1.367629e+02,1.996581e+00,1.167943e+01
std,4.418298e+01,4.400786e+01,3.499397e+00,1.626092e+01,2.110273e+00,5.991162e-02,6.988713e+01,6.132817e-02,1.626092e+01
min,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,4.000000e+00,2.002000e+03,0.000000e+00,5.000000e+00,8.800000e+01,2.000000e+00,5.000000e+00
50%,0.000000e+00,0.000000e+00,8.000000e+00,2.005000e+03,0.000000e+00,5.000000e+00,1.400000e+02,2.000000e+00,1.300000e+01
75%,7.000000e+00,8.000000e+00,1.000000e+01,2.013000e+03,0.000000e+00,5.000000e+00,1.790000e+02,2.000000e+00,1.600000e+01
max,2.625000e+03,2.635000e+03,1.200000e+01,2.018000e+03,1.260000e+02,6.000000e+00,5.630000e+02,3.000000e+00,2.018000e+03


In [6]:
manufacturers = clean_2018_df['MFR'].unique()
manufacturers.sort()
manufacturers

array(['AIR TRACTOR', 'AIRBUS', 'AMERICAN CHAMPION AIRCRAFT', 'AVEKO SRO',
       'BARD JOHN A', 'BENHAM JOHN', 'BETTS JOHN P', 'BIGHAM JACK L',
       'BOEING', 'BOMBARDIER INC', 'CANADAIR', 'CESSNA',
       'CHRISTOPHER LORD', 'CIRRUS DESIGN CORP', 'DASSAULT',
       'DIAMOND AIRCRAFT', 'DOUGLAS', 'EADS/ALENIA ATR', 'EMBRAER',
       'GLEASON BRENT L', 'GULFSTREAM AEROSPACE',
       'GULFSTREAM AMERICAN CORP.', 'MCDONNELL DOUGLAS', 'MOONEY',
       'PILATUS AIRCRAFT LTD', 'PIPER', 'QUEST AIRCRAFT COMPANY LLC',
       'ROBINSON', 'SAAB', 'SOCATA', 'TREMBLE JASON M'], dtype=object)